# System 1

In [26]:
import pandas as pd
import random

# ACX Dataset 

The dataset is the "Ground Truth" of this ACX Music system, its encyclopedia. Every time we want to get tracks or generate playlists, the dataset will be used as a reference point. The system is also built around a fixed amount of emotional categories. These categories are used to annotate the tracks in the dataset, by the user to rate tracks, and finally when we request playlists. The avaliable parameters are:
 
- amazement
- solemnity
- tenderness
- nostalgia 
- calmness
- power
- joyful_activation
- tension
- sadness

Let us first consider what our data actually looks like. Note how every track has an associated emotional annotation, average age and mood.  

<img src="data.jpg" alt="audio features" width="1000"/>

Aljanaki, A., Wiering, F., & Veltkamp, R. C. (2016). Studying emotion inducedby music through a crowdsourcing game. ,52(1), 115–128.  doi:  10.1016/j.ipm.2015.03.004

### Audio Feature Extraction 
For this sytem to work, we need extract and add audio features from the audio associated with each ```track id``` to our dataset. Let us, for now, simply add some random numbers to each row to illustrate how these audio features would be added and what it would look like in the dataset.

Why this process is necessary will become apparant later on in this demonstration.

<img src="spoof_features_with_emo - Copy.jpg" alt="audio features" width="1000"/>

# Lets dive inn!

In [30]:
import ACXmusic

# Create a user
Here we the users name and age. More information can also be logged here

In [31]:
user1 = ACXmusic.User('aleks', 'tidemann', 28)

# Then we log in
When we log in, we store general emotional information about the persons current state. In this prototype, we only log the users current ```mood``` (from bad (1) to good (5)) as well as the ```time of day``` and total number of ```log ins```.

Which parameters to log when the logging in can be adjusted to fit whatever ACX wants to track or use for playlist generation. How this ```mood``` parameter is used in this system will soon be apparant. 

In [32]:
user1.log_in(4) #Current user mood (from 1-5)

# User Info
Let us check some general user info to see how many times the user has used the app in the past and how many tracks is the user currently has rated.

In [33]:
user1.user_info()

'Full name - aleks tidemann\nAge - 28\nLogins - 3\nTracks rated - 4'

# Request a playlist
Now we ready to start the process of *requesting a playlist* based on the users current emtional state and his/hers desired emotional state. These states are referd to as ```From state``` and ```To state```. To generate a playlist, we first need to find tracks that correspond to our ```From state``` and to our ```To state```. Then, these tracks will be used to generate an actual playlist later on.

Let us start the process!

In [36]:
from_state = ["sadness"]
to_state = ["amazement", "joyful_activation"]

tracks = user1.get_tracks(from_state, to_state)
print("Done!")
print("Our {} tune is track nr.{} in the dataset".format(from_state, tracks[0]))
print("Our {} tune is track nr.{} in the dataset".format(to_state, tracks[1]))

get_tracks_matchCheck --> No 'from_state' matches found in user profile. Searching external database..
get_tracks_matchCheck_get_database_track --> More than one match found in the external database. Checking track associated moods to determine closest match.

get_tracks_matchCheck --> No 'to_state' matches found in user profile. Searching external database..
get_tracks_matchCheck_get_database_track --> One match found in external database.

Done!
Our ['sadness'] tune is track nr.176 in the dataset
Our ['amazement', 'joyful_activation'] tune is track nr.148 in the dataset


### What is actually happening here?

Why do we get these tracks as output?
Well, first we find all tracks in the **user profile** and compare them to the from/to state.
* If we find only ONE match, return it and we are done.
* If we find MORE than one match, we check the current ```mood``` of the user and return the track whose log-in mood corresponds best.

If we find NO matches in the user profile, we check the entire **external database** for matches:
* If we find only ONE match in the database, return it and we are done.
* If we find MORE than one match, we check the current mood of the user and return the track which mood corresponds best.

If we find NO matches in the database either, we check the **hamming distance** between the desired state and EVERY track in the database.
* If we find only ONE match in the database, return it and we are done.
* If more then one match is found, we again look at the users current ```mood``` to decide which track is closest.

It is also very much possible to not "travel" from one state to another. Simply pass the same emotional category in both states. 

# Linear Vector Interpolation
Next we would pass the "tracks" variable to a method called ```linear vector interpolation```. The method would first look for the tracks associated **audio features** in the dataset, as discussed previously and as shown in the illustration below.

<img src="spoof_features_with_emo%20-%20Copy.jpg" alt="audio features" width="1000"/>

Then, this method would **interpolate** between these audio features (vectors). The illustration below imagines a scenario where the user wants a playlist with 4 tracks. The illustration also only consideres 4 audio features instead of over 200 as in our actual dataset. Every red circle represents a track, a step along a high-dimensional "journey" twoards the users desired emotional state. 

<img src="linear-interpolation-w-imaginary-tracks.jpg" alt="Linear" width="600"/>

Effectively, these "steps" can be thought of as *imaginary tracks* because they have a collection of audio features which most likely doesnt correspond to any "real" audio file. 


But **WHY?!**
Let me explain..

# Machine Learning
Lastly, the *imaginary tracks* would be sent to a regressor algorithm (machine learning model). The model would then predict which tracks in the dataset **best fits** the *imaginary tracks* generated by the ```linear vector interpolation``` method above. In other words, the models goal would be to predict Track ID's based on fake audio features.


<img src="ml.jpg" alt="Linear" width="600"/>


As you can see, the output of the model is a list of tracks in the dataset, a playlist. Finally, the user would listen to these tracks sequentially and rate them.

# Rating of tracks 
When given a playlist, the user will rate each track according to the induced emotion he/she feels. The feedback will then be stored in the users profile. These preferences will then override the emotional parameters associated with the given track in the database, effectively generating a personalized dataset over time.  

example: 
- The database might have track nr.10 rated as being "sadness" and "nostalgic".
- However, if a user rates track nr.10 as being "joyful_activation" and "power", this rating will **override** the database's rating for this particular user. 

Let us rate some tracks!

In [ ]:
user_input = {40: ["amazement"]}
user1.set_track(user_input)

In [ ]:
user_input = {20: ["tension", "sadness"]}
user1.set_track(user_input)

In [ ]:
user_input = {64: ["joyful_activation", "tenderness", "nostalgia"]}
user1.set_track(user_input)

# User Profile
Lets see what the user profile actually looks like. Note track 64, 20 and 40, which we recently logged with various emotional categories.

In [ ]:
user1.user_data

# Log out 

In [ ]:
user1.log_out()